In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *
from SystematicsHelpers import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

import warnings
warnings.filterwarnings("ignore")

<h1>Configuration Stuff Here</h1>

In [ ]:
m = 200

savePath = "../plot_files/14April2024_m"+str(m)+"_v3_systematics/"

<h3>HNL</h3>

In [ ]:
file = open("../pkl_files/v3_April2024/df_postselect_weight_m"+str(m)+"_v3_hnl.pkl", 'rb')
df_hnl = pickle.load(file)
file.close()

<h3>TPC Neutrino</h3>

In [ ]:
file = open("../pkl_files/v3_April2024/df_postselect_weight_v3_nu.pkl", 'rb')
df_nu = pickle.load(file)
file.close()

<h3>Intime Cosmics</h3>

In [ ]:
file = open("../pkl_files/v3_April2024/df_postselect_weight_v3_cos.pkl", 'rb')
df_cos = pickle.load(file)
file.close()

#should be empty
print(df_cos)

In [ ]:
df_hnl = df_hnl.reset_index()
df_nu = df_nu.reset_index()
df_cos = df_cos.reset_index()

<h1>Do HNL Scaling</h1>

In [ ]:
df_hnl

In [ ]:
true_counts = df_hnl['true_counts'][0] 
start_counts = df_hnl['true_counts'][0]

simU = df_hnl['simU'].unique()[0]
plotU = df_hnl['plotU'].unique()[0]

scale = (plotU/simU)**2

hnl_scale_factor = df_hnl['scale_pot'].unique()[0]
sim_hnl_scale_factor = df_hnl['sim_scale_pot'].unique()[0]

print("Sim U = {}".format(simU))
print("Sim POT scale factor = {}".format(sim_hnl_scale_factor))
print("----------------------------------------------")
print("Sim to Plot scale by = {}".format(scale))
print("Plot U = {}".format(plotU))
print("Plot POT scale factor = {}".format(hnl_scale_factor))


scale = 1/100
hnl_scale_factor = hnl_scale_factor * scale
fitU = getUfromScaleFactor(plotU, scale)

print("----------------------------------------------")
print("Sim to Plot scale by = {}".format(scale))
print("Fit U = {}".format(fitU))
print("Fit POT scale factor = {}".format(hnl_scale_factor))

<h1>Separate Cosmics Sample </h1>

In [ ]:
df_cos_nu = df_nu[df_nu['slc_true_event_type'] == 9]
df_cos_hnl = df_hnl[df_hnl['slc_true_event_type'] == 9]

df_nu = df_nu[df_nu['slc_true_event_type'] != 9]
df_hnl = df_hnl[df_hnl['slc_true_event_type'] != 9]

In [ ]:
#keep_branches = [ "run", "subrun", "event", "slc_id", "slc_comp", "scale_pot", "mod_t"]
keep_branches = [ "scale_pot", "mod_t"]

df_cos_nu = df_cos_nu[keep_branches]
df_cos_hnl = df_cos_hnl[keep_branches]

print("Cosmics #events from Rockbox = {}".format(len(df_cos_nu)))
print("Cosmics #events from HNL = {}".format(len(df_cos_hnl)))

df_cos = pd.concat([df_cos_nu])#, df_cos_hnl])
print("Cosmics #events = {}".format(len(df_cos)))

<h3>Rockbox Cosmics</h3>

In [ ]:
even_bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
even_bins_mid = np.convolve(even_bins, [0.5, 0.5], "valid")

cos_true =[]
cos_true, _ = np.histogram(np.array(df_cos['mod_t']), bins=even_bins)

print(cos_true)
print(cos_true.sum())

#fill bins intergral = number of bins 
cos_flat = np.array([len(df_cos)/ 19] * 19)
print(cos_flat)
print(cos_flat.sum())

cos_flat_rebin = []

#rebin flat distribution into the chosen bins
for first, second in zip(bins, bins[1:]):
    #print(first, second)
    #print(cos_norm_cv[first:second])
    cos_flat_rebin.append(cos_flat[first:second].sum())   
    
cos_flat_rebin = np.array(cos_flat_rebin)
print(cos_flat_rebin)
print(cos_flat_rebin.sum())

#=================================================================#

fig, ax = plt.subplots(1,1, figsize = (6,4))
#-----------------------------------------------------------------#
ax.step(even_bins, np.insert(cos_true, 0, 0)
         , color = cos_col
         , label =  "True Cosmics"
)
#-----------------------------------------------------------------#
ax.step(even_bins, np.insert(cos_flat, 0, 0)
         , color = col_dict["Peach"]
         , label =  "Flat Cosmics"
)
#-----------------------------------------------------------------#
ax.step(bins, np.insert(cos_flat_rebin, 0, 0)
         , color = col_dict["Teal"]
         , label =  "Rebin Flat Cosmics"
)
#-----------------------------------------------------------------#
ax.legend(loc = 'upper left',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 2)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+"_flatten_and_rebin_cosmics", dpi=200)
plt.show()

In [ ]:
cos_true_hnl =[]
cos_true_hnl, _ = np.histogram(np.array(df_cos_hnl['mod_t']), bins=even_bins)

print(cos_true_hnl)
print(cos_true_hnl.sum())

#fill bins intergral = number of bins 
cos_flat_hnl = np.array([len(df_cos_hnl)/ 19] * 19)
print(cos_flat_hnl)
print(cos_flat_hnl.sum())

cos_flat_rebin_hnl = []

#rebin flat distribution into the chosen bins
for first, second in zip(bins, bins[1:]):
    cos_flat_rebin_hnl.append(cos_flat_hnl[first:second].sum())   
    
cos_flat_rebin_hnl = np.array(cos_flat_rebin_hnl)
print(cos_flat_rebin_hnl)
print(cos_flat_rebin_hnl.sum())

#=================================================================#

fig, ax = plt.subplots(1,1, figsize = (6,4))
#-----------------------------------------------------------------#
ax.step(even_bins, np.insert(cos_true_hnl, 0, 0)
         , color = cos_col
         , label =  "True Cosmics"
)
#-----------------------------------------------------------------#
ax.step(even_bins, np.insert(cos_flat_hnl, 0, 0)
         , color = col_dict["Peach"]
         , label =  "Flat Cosmics"
)
#-----------------------------------------------------------------#
ax.step(bins, np.insert(cos_flat_rebin_hnl, 0, 0)
         , color = col_dict["Teal"]
         , label =  "Rebin Flat Cosmics"
)
#-----------------------------------------------------------------#
ax.legend(loc = 'upper left',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 10)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+"_flatten_and_rebin_HNL_cosmics", dpi=200)
plt.show()

<h1>Separate Neutrinos sample</h1>

In [ ]:
unique_pot = df_nu['scale_pot'].unique()

rockbox_scale_factor = 1
ncpi0_scale_factor = 1
ccnue_scale_factor = 1

for p in unique_pot:
    if round(p) == 22:
        rockbox_scale_factor = p
    if round(p) == 4:
        ncpi0_scale_factor = p
    if round(p) == 1:
        ccnue_scale_factor = p

print("----------------------------------------------")
print("POT scale factor in the array = {}".format(unique_pot))
print("Rockbox = {}".format(rockbox_scale_factor))
print("NCPi0 = {}".format(ncpi0_scale_factor))
print("CCnue = {}".format(ccnue_scale_factor))
        
df_nu_rockbox = df_nu[df_nu['scale_pot'] ==  rockbox_scale_factor]
df_nu_ncpi0 = df_nu[df_nu['scale_pot'] ==  ncpi0_scale_factor]
df_nu_ccnue = df_nu[df_nu['scale_pot'] ==  ccnue_scale_factor]

print("----------------------------------------------")
print("total nu #events = {}".format(len(df_nu)))
print("rockbox #events = {}".format(len(df_nu_rockbox)))
print("ncpi0 #events = {}".format(len(df_nu_ncpi0)))
print("ccnue #events = {}".format(len(df_nu_ccnue)))

<h1> Organise Some Stuff</h1>

In [ ]:
hnl_dict = {}
hnl_cos_dict = {}

rockbox_dict = {}
ncpi0_dict = {}
ccnue_dict = {}

cos_dict = {}

cos_error_list = ['stat']
hnl_error_list = ['stat', 'flx', 'mistagging']
nu_error_list = ['stat', 'flx', 'xsec', 'g4']

In [ ]:
hnl_label = str(m) + ' MeV HNL ${\pi}^{0}$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(fitU,0,0))
nu_label = "Neutrinos"
rockbox_label = "Rockbox Neutrinos"
ncpi0_label = r"NC$\pi^{0}$"
ccnue_label = r"CC$\nu_{e}$"
cos_label = "Cosmics"

<h1>Plot Individual Sample</h1>

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
print(bins)
#-----------------------------------------------------------------#
pltdf = df_hnl['mod_t']
weights = [hnl_scale_factor]*len(df_hnl['mod_t']) 

_, _, _ = ax1.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = hnl_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = hnl_label
                        )

#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 12)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(1, 100)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+str("hnl_cv.png"), dpi=200)
    
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (18,4))

#-----------------------------------------------------------------#
pltdf = df_nu_rockbox['mod_t']
weights = df_nu_rockbox['scale_pot']
_, _, _ = ax1.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = nu_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = rockbox_label
                        )
#-----------------------------------------------------------------#
pltdf = df_nu_ncpi0['mod_t']
weights = df_nu_ncpi0['scale_pot']
_, _, _ = ax2.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = nu_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = ncpi0_label
                        )
#-----------------------------------------------------------------#
pltdf = df_nu_ccnue['mod_t']
weights = df_nu_ccnue['scale_pot']
_, _, _ = ax3.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = nu_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = ccnue_label
                        )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 12)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(1, 600)
#-----------------------------------------------------------------#
ax2.legend(loc = 'upper left',fontsize = 12)
plot_tick(ax2, 16)
plot_title(ax2, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(1, 1800)
#-----------------------------------------------------------------#
ax3.legend(loc = 'upper left',fontsize = 12)
plot_tick(ax3, 16)
plot_title(ax3, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax3.set_xlim(xmin, xmax)
ax3.set_ylim(1, 200)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+str("neutrino_cv.png"), dpi=200)
    
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (6,4))

#-----------------------------------------------------------------#
ax.step(bins, np.insert(cos_flat_rebin*rockbox_scale_factor, 0, 0)
         , color = cos_col
         , label =  "Cosmics"
)
#-----------------------------------------------------------------#
ax.legend(loc = 'upper left',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 2 * rockbox_scale_factor)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+str("cosmics_cv.png"), dpi=200)
plt.show()

<h1>Plot Overlay Sample</h1>

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#

pltdf = df_nu['mod_t']
weights = df_nu['scale_pot']
_, _, _ = ax1.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = nu_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = nu_label
                        )
#-----------------------------------------------------------------#
scale_up_just_once = 10

hnl_scale_up_just_once = hnl_scale_factor * scale_up_just_once
U_scale_up_just_once = getUfromScaleFactor(fitU, scale_up_just_once)

print("----------------------------------------------")
print("Sim to Plot scale by = {}".format(scale))
print("Fit U = {}".format(fitU))
print("Fit POT scale factor = {}".format(hnl_scale_factor))

this_hnl_label = str(m) + ' MeV HNL ${\pi}^{0}$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(U_scale_up_just_once,0,0))
#-----------------------------------------------------------------#
pltdf = df_hnl['mod_t']
weights = [hnl_scale_factor*10]*len(df_hnl['mod_t']) 

_, _, _ = ax1.hist(
                            pltdf,
                            bins = bins, 
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = hnl_col,
                            linestyle = "-",
                            linewidth = 2,
                            label = this_hnl_label
                        )
#-----------------------------------------------------------------#
ax1.step(bins, np.insert(cos_flat_rebin*rockbox_scale_factor, 0, 0)
         , color = cos_col
         , label =  "Cosmics"
)
#-----------------------------------------------------------------#

plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(1, 3000)

handles, labels = plt.gca().get_legend_handles_labels()
handles[-1] =extra = Rectangle((0, 0), 0.1, 0.1, fc="w", fill=False, edgecolor=cos_col, linewidth=2.0)

ax1.legend(handles, labels, loc = 'upper right',fontsize = 12)
#-----------------------------------------------------------------#
fig.tight_layout()

plt.savefig(savePath+str("signal_bkg_overlay_cv.png"), dpi=200)
    
plt.show()

<h1>HNL</h1>

<h3>Statistics</h3>

In [ ]:
#bin it
hnl_cv, _ = np.histogram(np.array(df_hnl['mod_t']), bins = bins)

#No Scale
print("\nprescale: entries per bin")
print(hnl_cv)

#Make covariance matrix
hnl_stat_cov = np.diag(hnl_cv) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
hnl_stat_err = np.sqrt(np.diag(hnl_stat_cov))

print("\n stat err")
print(hnl_stat_err)

#save in dictionary
hnl_cv_plot = np.insert(hnl_cv, 0, 0)

hnl_dict['cv'] = hnl_cv
hnl_dict['cv_plot'] = hnl_cv_plot

hnl_dict['stat_cov'] = hnl_stat_cov
hnl_dict['stat_err'] = hnl_stat_err

#plot and save
plot_hatchy_hatch(hnl_dict, hnl_label, "hnl", "stat_err")

plt.savefig(savePath+str("hnl_statistics_error.png"), dpi=200)
    
plt.show()

<h3>Flux</h3>

In [ ]:
#Make covariance matrix per variable
hnl_flx_cov_array =loopy_loop_multisim_universe(flux_list, flux_name, 1000, df_hnl, hnl_dict, 'hnl', savePath)

#Add them together
hnl_flx_cov = new_empty_cov()

for cov in hnl_flx_cov_array:
    hnl_flx_cov = hnl_flx_cov + cov
    
hnl_flx_err = np.sqrt(np.diag(hnl_flx_cov))

#save in dictionary
hnl_dict['flx_cov'] = hnl_flx_cov
hnl_dict['flx_err'] = hnl_flx_err

#plot it
plot_hatchy_hatch(hnl_dict, hnl_label, "hnl", "flx_err")

plt.savefig(savePath+str("hnl_flux_error.png"), dpi=200)
    
plt.show()

<h3>Cosmics As Mistagging Events</h3>

In [ ]:
#Make covariance matrix
hnl_mistagging_cov = np.outer(cos_flat_rebin_hnl, cos_flat_rebin_hnl)
hnl_mistagging_err = np.sqrt(np.diag(hnl_mistagging_cov))

print("\n Mistagging err")
print(hnl_mistagging_err)

#save in dictionary
hnl_dict['mistagging_cov'] = hnl_mistagging_cov
hnl_dict['mistagging_err'] = hnl_mistagging_err

#plot it
plot_hatchy_hatch(hnl_dict, hnl_label, "hnl", "mistagging_err")

plt.savefig(savePath+str("hnl_mistagging_error.png"), dpi=200)
    
plt.show()

<h3>Combine Errors</h3>

In [ ]:
combine_error(hnl_dict, hnl_error_list)

#plot it
plot_combine_err(hnl_dict, "hnl", hnl_label, hnl_error_list)

plt.savefig(savePath+str("hnl_beam_bucket_combined_covariance.png"), dpi=200)
    
plt.show()

<h3>Scale To POT AND Umu Coupling</h3>

In [ ]:
scale_cov_matrix(hnl_dict, hnl_scale_factor, hnl_error_list)

In [ ]:
plot_combine_err(hnl_dict, "hnl", hnl_label, hnl_error_list
                  , ifScale = True , scaleYmax = hnl_scale_factor, suffix = '_scale')

plt.savefig(savePath+str("hnl_beam_bucket_combined_covariance_scaled.png"), dpi=200)
    
plt.show()

<h1>Rockbox Neutrino</h1>

<h3>Statistics</h3>

In [ ]:
#bin it
rockbox_cv, _ = np.histogram(np.array(df_nu_rockbox['mod_t']), bins = bins)

rockbox_dict['cv'] = rockbox_cv
rockbox_dict['cv_plot'] =  np.insert(rockbox_cv, 0, 0)

rockbox_dict['stat_cov'] = np.diag(rockbox_cv) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
rockbox_dict['stat_err'] = np.sqrt(np.diag(rockbox_dict['stat_cov']))

#No Scale
print("\nprescale: entries per bin")
print(rockbox_dict['cv'] )

#===========================================================================#

#plot and save
plot_hatchy_hatch(rockbox_dict, rockbox_label, "rockbox", "stat_err")

plt.savefig(savePath+str("rockbox_stats_error.png"), dpi=200)
    
plt.show()

<h3>Flux</h3>

In [ ]:
rockbox_flx_cov_array =loopy_loop_multisim_universe(flux_list, flux_name, 1000
                                                    , df_nu_rockbox, rockbox_dict, 'rockbox'
                                                    , savePath)

#Add them together
rockbox_flx_cov = new_empty_cov()

for cov in rockbox_flx_cov_array:
    rockbox_flx_cov = rockbox_flx_cov + cov
    
rockbox_flx_err = np.sqrt(np.diag(rockbox_flx_cov))

rockbox_dict['flx_cov'] = rockbox_flx_cov
rockbox_dict['flx_err'] = rockbox_flx_err

#plot it

plot_hatchy_hatch(rockbox_dict, rockbox_label, "rockbox", "flx_err")

plt.savefig(savePath+str("rockbox_flux_error.png"), dpi=200)
    
plt.show()

<h3>Cross-Section: UniSim</h3>

In [ ]:
for name in unisim_list:
    df_nu_rockbox[name] = df_nu_rockbox[name].apply(lambda row: check_unisim(row))

In [ ]:
rockbox_unisim_cov_array = loopy_loop_unisim(df_nu_rockbox, rockbox_dict, "rockbox", savePath)

<h3>Cross-Section: Multi-Sigma</h3>

In [ ]:
#make a random gaussian gun
mu, sigma = 0, 1 # mean and standard deviation
n_univ = len(df_nu['slc_xsec_multisim_total'][0])

random_arr = np.random.normal(mu, sigma, n_univ)
len_univ = np.arange(0, n_univ)
plt.hist(random_arr)

In [ ]:
rockbox_multisigma_cov_arr = loopy_loop_multisigma(multisigma_list, multisigma_list, random_arr
                                                   , df_nu_rockbox, rockbox_dict, "rockbox"
                                                   , savePath
                                                  )

<h3>Cross-Section: Multi-Sim</h3>

In [ ]:
rockbox_multisim_cov_array =loopy_loop_multisim_universe(multisim_list, multisim_list, 500
                                                         , df_nu_rockbox, rockbox_dict, 'rockbox'
                                                         , savePath
                                                        )

<h3>Cross-Section: Combined</h3>

In [ ]:
print(len(rockbox_unisim_cov_array))
print(len(rockbox_multisigma_cov_arr))
print(len(rockbox_multisim_cov_array))

rockbox_xsec_cov_array = rockbox_unisim_cov_array + rockbox_multisigma_cov_arr + rockbox_multisim_cov_array
print(len(rockbox_xsec_cov_array))

rockbox_xsec_cov = new_empty_cov()

for cov in rockbox_xsec_cov_array:
    rockbox_xsec_cov = rockbox_xsec_cov + cov
    
rockbox_xsec_err = np.sqrt(np.diag(rockbox_xsec_cov))

rockbox_dict['xsec_cov'] = rockbox_xsec_cov
rockbox_dict['xsec_err'] = rockbox_xsec_err

In [ ]:
plot_hatchy_hatch(rockbox_dict, rockbox_label, "rockbox", "xsec_err")

plt.savefig(savePath+str("rockbox_xsec_error.png"), dpi=200)
    
plt.show()

<h3>Geant4 Re-Interactions</h3>

In [ ]:
rockbox_g4_cov = loopy_loop_multisim_universe(g4_list, g4_name, 1000, df_nu_rockbox, rockbox_dict, "rockbox", savePath)
rockbox_g4_cov = rockbox_g4_cov[0]

rockbox_g4_err = np.sqrt(np.diag(rockbox_g4_cov))

#save it
rockbox_dict['g4_cov'] = rockbox_g4_cov
rockbox_dict['g4_err'] = rockbox_g4_err
print(rockbox_g4_err)

#plot it
plot_hatchy_hatch(rockbox_dict, rockbox_label, "rockbox", "g4_err")

plt.savefig(savePath+str("rockbox_g4_error.png"), dpi=200)
    
plt.show()

<h3> Combine Errors </h3>

In [ ]:
combine_error(rockbox_dict, nu_error_list)

#plot it
plot_combine_err(rockbox_dict, "rockbox", rockbox_label, nu_error_list)

plt.savefig(savePath+str("rockbox_beam_bucket_combined_covariance.png"), dpi=200)
    
plt.show()

<h3> Scale to POT </h3>

In [ ]:
print(rockbox_scale_factor)

scale_cov_matrix(rockbox_dict, rockbox_scale_factor, nu_error_list)

#plot it
plot_combine_err(rockbox_dict, "rockbox", rockbox_label, nu_error_list
                  , ifScale = True , scaleYmax = rockbox_scale_factor, suffix ='_scale')

plt.savefig(savePath+str("rockbox_beam_bucket_combined_covariance_scaked.png"), dpi=200)
    
plt.show()

<h1>NCPi0</h1>

<h3>Statistics</h3>

In [ ]:
#bin it
ncpi0_cv, _ = np.histogram(np.array(df_nu_ncpi0['mod_t']), bins = bins)

ncpi0_dict['cv'] = ncpi0_cv
ncpi0_dict['cv_plot'] =  np.insert(ncpi0_cv, 0, 0)

ncpi0_dict['stat_cov'] = np.diag(ncpi0_cv) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
ncpi0_dict['stat_err'] = np.sqrt(np.diag(ncpi0_dict['stat_cov']))

#No Scale
print("\nprescale: entries per bin")
print(ncpi0_dict['cv'] )

#===========================================================================#

#plot and save
plot_hatchy_hatch(ncpi0_dict, ncpi0_label, "ncpi0", "stat_err")

plt.savefig(savePath+str("ncpi0_stats_error.png"), dpi=200)

plt.show()                                                                                                                                

<h3>Flux</h3>

In [ ]:
ncpi0_flx_cov_array =loopy_loop_multisim_universe(flux_list, flux_name, 1000, df_nu_ncpi0, ncpi0_dict, 'ncpi0', savePath)

#add cov matrix
ncpi0_flx_cov = new_empty_cov()

for cov in ncpi0_flx_cov_array:
    ncpi0_flx_cov = ncpi0_flx_cov + cov
    
ncpi0_flx_err = np.sqrt(np.diag(ncpi0_flx_cov))

ncpi0_dict['flx_cov'] = ncpi0_flx_cov
ncpi0_dict['flx_err'] = ncpi0_flx_err

#plot it

plot_hatchy_hatch(ncpi0_dict, ncpi0_label, "ncpi0", "flx_err")

plt.savefig(savePath+str("ncpi0_flx_err.png"), dpi=200)

<h3>Cross-Section: UniSim</h3>

In [ ]:
for name in unisim_list:
    
    df_nu_ncpi0[name] = df_nu_ncpi0[name].apply(lambda row: check_unisim(row))

In [ ]:
ncpi0_unisim_cov_array = loopy_loop_unisim(df_nu_ncpi0, ncpi0_dict, "ncpi0", savePath)

<h3>Cross-Section: Multi-Sigma</h3>

In [ ]:
ncpi0_multisigma_cov_arr = loopy_loop_multisigma(multisigma_list, multisigma_list, random_arr
                                                 , df_nu_ncpi0, ncpi0_dict, "ncpi0"
                                                 , savePath
                                                )

<h3>Cross-Section: Multi-Sim</h3>

In [ ]:
ncpi0_multisim_cov_array =loopy_loop_multisim_universe(multisim_list, multisim_list, 500
                                                       , df_nu_ncpi0, ncpi0_dict, 'ncpi0'
                                                       , savePath
                                                      )

<h3>Cross-Section: Combined</h3>

In [ ]:
print(len(ncpi0_unisim_cov_array))
print(len(ncpi0_multisigma_cov_arr))
print(len(ncpi0_multisim_cov_array))

ncpi0_xsec_cov_array = ncpi0_unisim_cov_array  + ncpi0_multisigma_cov_arr + ncpi0_multisim_cov_array
print(len(ncpi0_xsec_cov_array))

ncpi0_xsec_cov = new_empty_cov()

for cov in ncpi0_xsec_cov_array:
    ncpi0_xsec_cov = ncpi0_xsec_cov + cov
    
ncpi0_xsec_err = np.sqrt(np.diag(ncpi0_xsec_cov))

ncpi0_dict['xsec_cov'] = ncpi0_xsec_cov
ncpi0_dict['xsec_err'] = ncpi0_xsec_err

print(ncpi0_xsec_err)

#plot it
plot_hatchy_hatch(ncpi0_dict, ncpi0_label, "ncpi0", "xsec_err")

plt.savefig(savePath+str("ncpi0_xsec_err.png"), dpi=200)

<h3>Geant4 Re-Interactions</h3>

In [ ]:
ncpi0_g4_cov = loopy_loop_multisim_universe(g4_list, g4_name, 1000, df_nu_ncpi0, ncpi0_dict, "ncpi0", savePath)
ncpi0_g4_cov = ncpi0_g4_cov[0]

ncpi0_g4_err = np.sqrt(np.diag(ncpi0_g4_cov))

ncpi0_dict['g4_cov'] = ncpi0_g4_cov
ncpi0_dict['g4_err'] = ncpi0_g4_err
print(ncpi0_g4_err)

#plot it
plot_hatchy_hatch(ncpi0_dict, ncpi0_label, "ncpi0", "g4_err")

plt.savefig(savePath+str("ncpi0_g4_error.png"), dpi=200)
    
plt.show()

<h3>Combine Errors</h3>

In [ ]:
combine_error(ncpi0_dict, nu_error_list)

#plot it
plot_combine_err(ncpi0_dict, "ncpi0", ncpi0_label, nu_error_list)

plt.savefig(savePath+str("ncpi0_beam_bucket_combined_covariance.png"), dpi=200)
    
plt.show()

<h3>Scale to POT </h3>

In [ ]:
print(ncpi0_scale_factor)

scale_cov_matrix(ncpi0_dict, ncpi0_scale_factor, nu_error_list)

#plot it
plot_combine_err(ncpi0_dict, "ncpi0", ncpi0_label, nu_error_list
                 , ifScale = True, scaleYmax = ncpi0_scale_factor, suffix ='_scale')

plt.savefig(savePath+str("ncpi0_beam_bucket_combined_covariance_scaked.png"), dpi=200)
    
plt.show()

<h1>CCNue</h1>

<h3>Statistics</h3>

In [ ]:
#bin it
ccnue_cv, _ = np.histogram(np.array(df_nu_ccnue['mod_t']), bins = bins)

ccnue_dict['cv'] = ccnue_cv
ccnue_dict['cv_plot'] =  np.insert(ccnue_cv, 0, 0)

ccnue_dict['stat_cov'] = np.diag(ccnue_cv) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
ccnue_dict['stat_err'] = np.sqrt(np.diag(ccnue_dict['stat_cov']))

#No Scale
print("\nprescale: entries per bin")
print(ccnue_dict['cv'] )

#===========================================================================#

#plot and save
plot_hatchy_hatch(ccnue_dict, ccnue_label, "ccnue", "stat_err")

plt.savefig(savePath+str("ccnue_stats_error.png"), dpi=200)

plt.show()

<h3>Flux</h3>

In [ ]:
ccnue_flx_cov_array =loopy_loop_multisim_universe(flux_list, flux_name, 1000, df_nu_ccnue, ccnue_dict, 'ccnue', savePath)

#add cov matrix
ccnue_flx_cov = new_empty_cov()

for cov in ccnue_flx_cov_array:
    ccnue_flx_cov = ccnue_flx_cov + cov
    
ccnue_flx_err = np.sqrt(np.diag(ccnue_flx_cov))

ccnue_dict['flx_cov'] = ccnue_flx_cov
ccnue_dict['flx_err'] = ccnue_flx_err

#plot it

plot_hatchy_hatch(ccnue_dict, ccnue_label, "ccnue", "flx_err")

plt.savefig(savePath+str("ccnue_flx_err.png"), dpi=200)


<h3>Cross-Section: Unisim</h3>

In [ ]:
for name in unisim_list:
    
    df_nu_ccnue[name] = df_nu_ccnue[name].apply(lambda row: check_unisim(row))

In [ ]:
ccnue_unisim_cov_array = loopy_loop_unisim(df_nu_ccnue, ccnue_dict, "ccnue", savePath)

<h3>Cross-Section: Multi-Sigma</h3>

In [ ]:
ccnue_multisigma_cov_arr = loopy_loop_multisigma(multisigma_list, multisigma_list, random_arr
                                                 , df_nu_ccnue, ccnue_dict, "ccnue"
                                                 , savePath
                                                )

<h3>Cross-Section: Multi-Sim</h3>

In [ ]:
ccnue_multisim_cov_array =loopy_loop_multisim_universe(multisim_list, multisim_list, 500
                                                       , df_nu_ccnue, ccnue_dict, 'ccnue'
                                                       , savePath
                                                      )

<h3>Cross-Section: Combined</h3>

In [ ]:
print(len(ccnue_unisim_cov_array))
print(len(ccnue_multisigma_cov_arr))
print(len(ccnue_multisim_cov_array))

ccnue_xsec_cov_array = ccnue_unisim_cov_array  + ccnue_multisigma_cov_arr + ccnue_multisim_cov_array
print(len(ccnue_xsec_cov_array))

ccnue_xsec_cov = new_empty_cov()

for cov in ccnue_xsec_cov_array:
    ccnue_xsec_cov = ccnue_xsec_cov + cov
    
ccnue_xsec_err = np.sqrt(np.diag(ccnue_xsec_cov))

ccnue_dict['xsec_cov'] = ccnue_xsec_cov
ccnue_dict['xsec_err'] = ccnue_xsec_err

print(ccnue_xsec_err)

#plot it
plot_hatchy_hatch(ccnue_dict, ccnue_label, "ccnue", "xsec_err")

plt.savefig(savePath+str("ccnue_xsec_err.png"), dpi=200)


<h3>Geant4 Re-Interactions</h3>

In [ ]:
ccnue_g4_cov = loopy_loop_multisim_universe(g4_list, g4_name, 1000, df_nu_ccnue, ccnue_dict, "ccnue", savePath)
ccnue_g4_cov = ccnue_g4_cov[0]

ccnue_g4_err = np.sqrt(np.diag(ccnue_g4_cov))

ccnue_dict['g4_cov'] = ccnue_g4_cov
ccnue_dict['g4_err'] = ccnue_g4_err
print(ccnue_g4_err)

#plot it
plot_hatchy_hatch(ccnue_dict, ccnue_label, "ccnue", "g4_err")

plt.savefig(savePath+str("ccnue_g4_error.png"), dpi=200)
    
plt.show()


<h3>Combine Errors</h3>

In [ ]:
combine_error(ccnue_dict, nu_error_list)

#plot it
plot_combine_err(ccnue_dict, "ccnue", ccnue_label, nu_error_list)

plt.savefig(savePath+str("ccnue_beam_bucket_combined_covariance.png"), dpi=200)
    
plt.show()

<h3>Scale to POT </h3>

In [ ]:
print(ccnue_scale_factor)

scale_cov_matrix(ccnue_dict, ccnue_scale_factor, nu_error_list)

#plot it
plot_combine_err(ccnue_dict, "ccnue", ccnue_label, nu_error_list
                 , ifScale = True, scaleYmax = ccnue_scale_factor, suffix ='_scale')

plt.savefig(savePath+str("ccnue_beam_bucket_combined_covariance_scaked.png"), dpi=200)
    
plt.show()

<h1>Fuse Neutrino Background Into One Please</h1>

In [ ]:
nu_dict_list = [rockbox_dict, ncpi0_dict, ccnue_dict]

In [ ]:
nu_dict = {}

nu_dict['cv_scale'] = np.array([0]*(len(bins)-1))
nu_dict['cv_plot_scale'] = np.array([0]*(len(bins)))

for d in nu_dict_list:
    nu_dict['cv_scale'] = nu_dict['cv_scale'] + d['cv_scale']
    nu_dict['cv_plot_scale'] = nu_dict['cv_plot_scale'] + d['cv_plot_scale']

In [ ]:
fill_nan_plz(rockbox_dict)
fill_nan_plz(ncpi0_dict)
fill_nan_plz(ccnue_dict)

In [ ]:
nu_dict['combined_cov_scale'] = new_empty_cov()

for err in nu_error_list:
    print(err)
    nu_dict[err+'_cov_scale'] = new_empty_cov()
    
    for d in nu_dict_list:
        nu_dict[err+'_cov_scale'] = nu_dict[err+'_cov_scale'] + d[err+'_cov_scale']
    
    nu_dict[err+'_err_scale'] = np.sqrt(np.diag(nu_dict[err+'_cov_scale']))
    
    nu_dict[err+'_cov_frac_scale'] = nu_dict[err+'_cov_scale'] / np.outer(nu_dict['cv_scale'], nu_dict['cv_scale'])
    nu_dict[err+'_frac_err_scale'] = np.sqrt(np.diag(nu_dict[err+'_cov_frac_scale']))
        
    nu_dict['combined_cov_scale'] = nu_dict['combined_cov_scale'] + nu_dict[err+'_cov_scale']
    
nu_dict['combined_err_scale'] = np.sqrt(np.diag( nu_dict['combined_cov_scale']))
    
nu_dict['combined_cov_frac_scale'] = nu_dict['combined_cov_scale'] / np.outer(nu_dict['cv_scale'], nu_dict['cv_scale'])
nu_dict['combined_frac_err_scale'] = np.sqrt(np.diag(nu_dict['combined_cov_frac_scale']))

In [ ]:
plot_combine_err(nu_dict, "nu", "Neutrinos", nu_error_list
                  , ifScale = True, suffix = '_scale')

plt.savefig(savePath+str("combined_neutrino_beam_bucket_combined_covariance_scaled.png"), dpi=200)
    
plt.show()

<h1> Cosmics </h1>

<h3>Statistics</h3>

In [ ]:
print(cos_flat_rebin)

cos_dict['cv'] = cos_flat_rebin
cos_dict['cv_plot'] = np.insert(cos_dict['cv'], 0, 0)

#This is the smart way
cos_stat_cov = np.diag(cos_flat_rebin) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
cos_stat_err = np.sqrt(np.diag(cos_stat_cov))

print("\n stat err")
print(cos_stat_err)

cos_dict['stat_cov'] = cos_stat_cov
cos_dict['stat_err'] = cos_stat_err

#plot it
plot_hatchy_hatch(cos_dict, "Cosmics", "cos", "stat_err")

plt.savefig(savePath+str("cosmic_stat_err.png"), dpi=200)

<h3>Scale to POT to Rockbox Number </h3>

In [ ]:
combine_error(cos_dict, cos_error_list)

scale_cov_matrix(cos_dict, rockbox_scale_factor, cos_error_list)

In [ ]:
plot_combine_err(cos_dict, "cos", cos_label, cos_error_list, ifScale = True, scaleYmax = 25, suffix = '_scale')

plt.savefig(savePath+str("combined_cosmics_err_scaled.png"), dpi=200)

<h1>Fuse Neutrino and Cosmics Into A Single Background</h1>

In [ ]:
fill_nan_plz(cos_dict)
fill_nan_plz(nu_dict)

In [ ]:
bkg_dict = {}

#combine cv
bkg_dict['cv_scale'] = nu_dict['cv_scale'] + cos_dict['cv_scale'] 
bkg_dict['cv_plot_scale'] = nu_dict['cv_plot_scale'] + cos_dict['cv_plot_scale'] 

#combine stat errors
bkg_dict['stat_cov_scale'] = nu_dict['stat_cov_scale'] + cos_dict['stat_cov_scale']

bkg_dict['stat_err_scale'] = np.sqrt(np.diag(bkg_dict['stat_cov_scale']))

bkg_dict['stat_cov_frac_scale'] = bkg_dict['stat_cov_scale'] / np.outer(bkg_dict['cv_scale'], bkg_dict['cv_scale'])
bkg_dict['stat_frac_err_scale'] = np.sqrt(np.diag(bkg_dict['stat_cov_frac_scale']))

#flux / xsec / g4 are taken directly from neutrino samples

bkg_dict['flx_err_scale'] = nu_dict['flx_frac_err_scale']
bkg_dict['xsec_err_scale'] = nu_dict['xsec_frac_err_scale']
bkg_dict['g4_err_scale'] = nu_dict['g4_frac_err_scale']

bkg_dict['flx_frac_err_scale'] = nu_dict['flx_frac_err_scale']
bkg_dict['xsec_frac_err_scale'] = nu_dict['xsec_frac_err_scale']
bkg_dict['g4_frac_err_scale'] = nu_dict['g4_frac_err_scale']

#quadrature sum each uncertainty
bkg_dict['combined_err_scale'] = np.array([0]*(len(bins)-1))
bkg_dict['combined_frac_err_scale'] = np.array([0]*(len(bins)-1))

nu_error_list = ['stat','flx','xsec', 'g4']

for err in nu_error_list:
    print(err)
    bkg_dict['combined_err_scale'] = bkg_dict['combined_err_scale'] + bkg_dict[err+'_err_scale']**2
    bkg_dict['combined_frac_err_scale'] = bkg_dict['combined_frac_err_scale'] + + bkg_dict[err+'_frac_err_scale']**2
    
bkg_dict['combined_err_scale'] = np.sqrt(bkg_dict['combined_err_scale'])
bkg_dict['combined_frac_err_scale'] = np.sqrt(bkg_dict['combined_frac_err_scale'])

In [ ]:
plot_combine_err(bkg_dict, "nu", "Neutrino + Cosmic Background"
                 , nu_error_list
                 , ifScale = True, scaleYmax = 1.2, suffix = '_scale')

plt.savefig(savePath+str("full_background_err.png"), dpi=200)

<h1>Save HNL </h1>

In [ ]:
hnl_dict_save = {}

In [ ]:
hnl_dict_save['U'] = fitU

hnl_dict_save['m'] = m

hnl_dict_save['signal'] = hnl_dict['cv_scale'].tolist()

hnl_dict_save['stat_err'] = hnl_dict['stat_err_scale'].tolist()

hnl_dict_save['flx_err'] = hnl_dict['flx_err_scale'].tolist()

hnl_dict_save['mistagging_err'] = hnl_dict['mistagging_err_scale'].tolist()

hnl_dict_save['combined_err'] = hnl_dict['combined_err_scale'].tolist()

In [ ]:
for k,v in zip(hnl_dict_save.keys(), hnl_dict_save.values()):
    print(k, v)

In [ ]:
np.save("../pkl_files/v3_April2024/hnl_m"+str(m)+"_v3_dict.npy", hnl_dict_save) 

<h1>Save Neutrinos + Cosmics Background </h1>

In [ ]:
bkg_save_dict = {}

bkg_save_dict['bkg'] = bkg_dict['cv_scale'].tolist()

bkg_save_dict['stat_err'] = bkg_dict['stat_err_scale'].tolist()

bkg_save_dict['flx_err'] = bkg_dict['flx_err_scale'].tolist()

bkg_save_dict['xsec_err'] = bkg_dict['xsec_err_scale'].tolist()

bkg_save_dict['g4_err'] = bkg_dict['g4_err_scale'].tolist()

bkg_save_dict['combined_err'] = bkg_dict['combined_err_scale'].tolist()

In [ ]:
for k,v in zip(bkg_save_dict.keys(), bkg_save_dict.values()):
    print(k, v)

In [ ]:
np.save("../pkl_files/v3_April2024/bkg_v3_dict.npy", bkg_save_dict) 

<h1>Save Neutrinos Background </h1>

In [ ]:
nu_save_dict = {}

nu_save_dict['bkg'] = nu_dict['cv_scale'].tolist()

nu_save_dict['stat_err'] = nu_dict['stat_err_scale'].tolist()

nu_save_dict['flx_err'] = nu_dict['flx_err_scale'].tolist()

nu_save_dict['xsec_err'] = nu_dict['xsec_err_scale'].tolist()

nu_save_dict['g4_err'] = nu_dict['g4_err_scale'].tolist()

nu_save_dict['combined_err'] = nu_dict['combined_err_scale'].tolist()


In [ ]:
for k,v in zip(bkg_save_dict.keys(), bkg_save_dict.values()):
    print(k, v)

In [ ]:
np.save("../pkl_files/v3_April2024/nu_v3_dict.npy", nu_save_dict) 

<h1>Save Rockbox Cosmics Background </h1>

In [ ]:
cos_save_dict = {}

cos_save_dict['bkg'] = cos_dict['cv_scale'].tolist()

cos_save_dict['stat_err'] = cos_dict['stat_err_scale'].tolist()

In [ ]:
for k,v in zip(cos_save_dict.keys(), cos_save_dict.values()):
    print(k, v)

In [ ]:
np.save("../pkl_files/v3_April2024/cos_v3_dict.npy", cos_save_dict) 

###### 